In [ ]:
#Check GPU
!nvidia-smi

#import OS and build home 
import os
HOME = os.getcwd()
print(HOME)

# Pip install method (recommended)
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

#import YOLO from Ultralytics
from ultralytics import YOLO

from IPython.display import display, Image

#Create dataset folder and load annotated dataset from my own roboflow workspace
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="OcQy6FV5r1iF4mI6kpw1") #match API key 
project = rf.workspace("kmitl-lqtvt").project("jetson-the-rock-detector") #Project is workspace that this version of dataset is in
dataset = project.version(6).download("yolov8") #the dataset is the dataset version 6 from Roboflow 

#Use this command to train the dataset
%cd {HOME}

!yolo task=detect mode=train model=yolov8l.pt data='/content/datasets/Jetson-the-rock-(detector)-6/data.yaml' epochs=30 imgsz=640 plots=True batch=22

#Deploy this version to Roboflow project workspace
project.version(dataset.version).deploy(model_type="yolov8", model_path="/content/runs/detect/train3/")

In [ ]:
#install important function by using Python package manager(pip)
!pip3 install opencv-python numpy requests #in this case we will install Opencv numpy and ruquest, which we will use in the program to open camera

#import important function cv2(Opencv) base64 numpy requests
import cv2
import base64
import numpy as np
import requests

#accuire Roboflow API key from Roboflow user page
ROBOFLOW_API_KEY = "OcQy6FV5r1iF4mI6kpw1"
ROBOFLOW_MODEL = "FILL YOUR MODEL ID HERE" # eg xx-xxxx--#
ROBOFLOW_SIZE = 416

#use join method to create usable url
upload_url = "".join([
    "https://detect.roboflow.com/",
    ROBOFLOW_MODEL,
    "?access_token=",
    ROBOFLOW_API_KEY,
    "&format=image",
    "&stroke=5"
])

video = cv2.VideoCapture(0)

# Infer via the Roboflow Infer API
def infer():
    # Get the current image from the webcam
    ret, img = video.read()

    # Resize (while maintaining the aspect ratio) to improve speed and save bandwidth
    height, width, channels = img.shape
    scale = ROBOFLOW_SIZE / max(height, width)
    img = cv2.resize(img, (round(scale * width), round(scale * height)))

    # Encode image to base64 string
    retval, buffer = cv2.imencode('.jpg', img)
    img_str = base64.b64encode(buffer)

    # Get prediction from Roboflow Infer API
    resp = requests.post(upload_url, data=img_str, headers={
        "Content-Type": "application/x-www-form-urlencoded"
    }, stream=True).raw

    # Parse result image
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    return image

# Main loop; infers sequentially until you press "q"
while 1:
    # On "q" keypress, exit
    if(cv2.waitKey(1) == ord('q')):
        break

    # Synchronously get a prediction from the Roboflow Infer API
    image = infer()
    # And display the inference results
    cv2.imshow('image', image)

# Release resources when finished
video.release()
cv2.destroyAllWindows()